# 데이터 처리 

이 노트북은 `TigerGraph`에 저장된 그래프에서 일반적인 데이터 처리 작업에 `pyTigerGraph`를 사용하는 방법을 보여줍니다.

**NOTE**: 현재 ML Workbench에서 제공하는 모든 기능을 사용하려면 데이터베이스를 한 번만 활성화(activate)해야 합니다. ML Workbench on Cloud를 사용하는 경우 활성기가 포함되며 아래 셀을 실행(먼저 주석 해제)하여 활성화할 수 있습니다. 다른 버전의 Workbench의 경우 https://act.tigergraphlabs.com 에서 activator를 다운로드할 수 있습니다. 자세한 지침은 해당 웹사이트에도 포함되어 있습니다.

In [ ]:
# 아래 주석을 제거하고 필요한 정보를 입력하십시오. 자세한 지침은 https://act.tigergraphlabs.com을 참조하십시오.
# !mlwb activate [database address] -u [username] -p [password] -s [secret]

### 데이터베이스 연결 

이 `TigerGraphConnection` 클래스는 TigerGraph 데이터베이스에 대한 연결을 나타냅니다. 후드 아래에는 데이터베이스와 통신하는 데 필요한 정보가 저장됩니다. 꽤 많은 데이터베이스 작업을 수행할 수 있습니다. 자세한 내용은 해당 [문서](https://docs.tigergraph.com/pytigergraph/current/intro/) 를 참조하십시오.

**Note**: 2022년 7월 5일 이후에 생성된 TG 클라우드 DB는 사용자 username/password 대신 secret가 필요합니다. 그렇지 않으면 비워 둘 수 있습니다.


In [1]:
from pyTigerGraph import TigerGraphConnection

In [2]:
conn = TigerGraphConnection(
    host="http://127.0.0.1", # 데이터베이스 서버의 주소로 변경
    graphname="Cora",
    username="tigergraph",
    password="tigergraph",
    gsqlSecret="" # 2022년 7월 5일 이후에 생성된 TG 클라우드 DB에는 user/pass 대신 secret이 필요합니다.  
)

<span style="color:red">아래 셀의 주석을 제거하고 토큰 인증이 활성화된 경우 토큰을 가져오고 설정하기 위해 실행합니다.</span>. 
* 이것은 tgcloud의 모든 데이터베이스에 필요합니다.
* `<secret>`은 사용자 암호입니다. 자세한 내용은 https://docs.tigergraph.com/tigergraph-server/current/user-access/managing-credentials#_secrets 를 참조하세요.
* secret을 모르는 경우  `secret=conn.createSecret()`를 사용하여 secret을 만들 수 있습니다.

In [ ]:
#conn.getToken(<secret>)

In [3]:
# 그래프 스키마 및 기타 정보.
print(conn.gsql("ls"))

---- Graph Cora
Vertex Types:
- VERTEX Paper(PRIMARY_ID id INT, x LIST<INT>, y INT, train_mask BOOL, val_mask BOOL, test_mask BOOL) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
Edge Types:
- DIRECTED EDGE Cite(FROM Paper, TO Paper, time INT, is_train BOOL, is_val BOOL)

Graphs:
- Graph Cora(Paper:v, Cite:e)
Jobs:
- CREATE LOADING JOB load_cora_data FOR GRAPH Cora {
DEFINE FILENAME edge_csv = "./edges.csv";
DEFINE FILENAME node_csv = "./nodes.csv";
LOAD node_csv TO VERTEX Paper VALUES($"id", SPLIT($"x", " "), $"y", $"train", $"valid", $"test") USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD edge_csv TO EDGE Cite VALUES($"source", $"target", _, _, _) USING SEPARATOR=",", HEADER="true", EOL="\n";
}

- CREATE SCHEMA_CHANGE JOB Cora_job FOR GRAPH Cora {
ADD VERTEX Paper(PRIMARY_ID id INT, x LIST<INT>, y INT, train_mask BOOL, val_mask BOOL, test_mask BOOL) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true";
ADD DIRECTED EDGE Cite(FROM Paper, TO Paper,

In [4]:
# 모든 정점 유형에 대한 정점 수
conn.getVertexCount('*')

{'Paper': 2708}

In [5]:
# 특정 유형의 정점 수
conn.getVertexCount("Paper")

2708

In [6]:
# 모든 유형의 가장자리 수
conn.getEdgeCount()

{'Cite': 10556}

In [7]:
# 특정 유형의 간선 수
conn.getEdgeCount("Cite")

10556

### 무작위 정점 분할(Random Vertex Split)

`VertexSplitter`는 무작위로 정점을 최대 3개의 부분으로 분할합니다. 분할 결과는 제공된 정점 속성에 저장됩니다. 각 부울 속성은 정점이 속한 부분을 나타냅니다. 예를 들어 정점을 80% 훈련, 10% 검증 및 10% 테스트로 분할하려는 경우 `train_mask=0.8, val_mask=0.1, test_mask=0.1`로 splitter에 인수로 제공할 수 있습니다. 3개의 속성 `train_mask`, `val_mask`, `test_mask`는 그래프에 존재해야 합니다. 정점의 80%는 `train_mask=True`, 10%는 `val_mask=True`, 10%는 `test_mask=True` 무작위로 설정됩니다. 파티션 간에 겹치는 부분이 없습니다.

In [8]:
%%time
split = conn.gds.vertexSplitter(train_mask=0.8, val_mask=0.1, test_mask=0.1)


Installing and optimizing queries. It might take a minute if this is the first time you use this loader.
Query installation finished.
CPU times: user 309 ms, sys: 201 ms, total: 509 ms
Wall time: 2min 2s


In [9]:
%%time
split.run()

Splitting vertices...
Vertex split finished successfully.
CPU times: user 4.86 ms, sys: 3.05 ms, total: 7.91 ms
Wall time: 62.8 ms


이제 분할이 완료되었습니다. 모든 정점을 로드하고 분할이 올바른지 확인합니다. `VertexLoader`및 기타 데이터 로더 에 대한 자세한 내용은 다음 튜토리얼을 참조하세요 .

In [10]:
%%time
vertex_loader = conn.gds.vertexLoader(attributes=["train_mask", "val_mask", "test_mask"])

Installing and optimizing queries. It might take a minute if this is the first time you use this loader.
Query installation finished.
CPU times: user 5.47 ms, sys: 3.94 ms, total: 9.41 ms
Wall time: 28.1 s


In [11]:
%%time
data = vertex_loader.data

CPU times: user 19 ms, sys: 1.25 ms, total: 20.2 ms
Wall time: 44.5 ms


In [12]:
data.train_mask.sum()/len(data), data.val_mask.sum()/len(data), data.test_mask.sum()/len(data)

(0.7950516986706057, 0.10118168389955687, 0.10376661742983752)

### 무작위 간선 분할(Random Edge Split)

`EdgeSplitter`는 무작위로 에지를 최대 3개의 부분으로 분할합니다. 분할 결과는 제공된 에지 속성에 저장됩니다. 각 부울 속성은 간선이 속한 부분을 나타냅니다. 예를 들어 에지를 80% 훈련과 20% 검증으로 분할하려는 경우 `is_train=0.8, is_val=0.2`를 splitter에 인수로 제공할 수 있습니다. 2개의 속성 `is_train`, `is_val`이 그래프에 존재해야 합니다. 간선의 80%는 `is_train=True`로 설정되고 20%는 무작위로 `is_val=True`로 설정됩니다. 파티션 간에 겹치는 부분이 없습니다.

In [13]:
%%time
splitter = conn.gds.edgeSplitter(is_train=0.8, is_val=0.2)

Installing and optimizing queries. It might take a minute if this is the first time you use this loader.
Query installation finished.
CPU times: user 6.87 ms, sys: 3.78 ms, total: 10.7 ms
Wall time: 28.8 s


In [14]:
%%time
splitter.run()

Splitting edges...
Edge split finished successfully.
CPU times: user 4.56 ms, sys: 247 µs, total: 4.81 ms
Wall time: 82.3 ms


이제 분할이 완료되었습니다. 모든 에지를 로드하고 분할이 올바른지 확인합니다. `EdgeLoader` 및 기타 데이터 로더 에 대한 자세한 내용은 다음 x튜토리얼을 참조하세요 .

In [15]:
%%time
edge_loader = conn.gds.edgeLoader(attributes=["is_train", "is_val"])

Installing and optimizing queries. It might take a minute if this is the first time you use this loader.
Query installation finished.
CPU times: user 7.02 ms, sys: 3.56 ms, total: 10.6 ms
Wall time: 32.8 s


In [16]:
%%time
data = edge_loader.data

CPU times: user 11.3 ms, sys: 1.69 ms, total: 12.9 ms
Wall time: 37.3 ms


In [17]:
data.is_train.sum()/len(data), data.is_val.sum()/len(data)

(0.8020083364910952, 0.19799166350890487)